# Clustering Crypto

In [40]:
# Initial imports
import pandas as pd
import hvplot
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

### Deliverable 1: Preprocessing the Data for PCA

In [41]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col = 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [42]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading']==True]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [43]:
# Keep all the cryptocurrencies that have a working algorithm.
pd.isna(crypto_df['Algorithm'])

42      False
365     False
404     False
611     False
808     False
        ...  
SERO    False
UOS     False
BDX     False
ZEN     False
XBC     False
Name: Algorithm, Length: 1144, dtype: bool

In [45]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [47]:
# Remove rows that have at least 1 null value.
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [49]:
#remove rows
crypto_df=crypto_df.dropna()
crypto_df.count()

CoinName           685
Algorithm          685
ProofType          685
TotalCoinsMined    685
TotalCoinSupply    685
dtype: int64

In [50]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df.TotalCoinsMined > 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [51]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coins_name_df = crypto_df[['CoinName']].set_index([crypto_df.index])
coins_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [52]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], inplace=True)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [54]:
# Use get_dummies() to create variables for text features.
crypto = pd.get_dummies(crypto_df['Algorithm'])
dummy = pd.get_dummies(crypto_df['ProofType'])
combined = pd.concat([crypto,dummy], axis= 1)
df = crypto_df.merge(combined,left_index = True,right_index = True)
df = df.drop(['Algorithm','ProofType'], axis =1)
df

,TotalCoinsMined,TotalCoinSupply,1GB AES Pattern Search,536,Argon2d,BLAKE256,Blake,Blake2S,Blake2b,C11,...,PoW/PoS,PoW/PoS,PoW/PoW,PoW/nPoS,Pos,Proof of Authority,Proof of Trust,TPoS,Zero-Knowledge Proof,dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# Standardize the data with StandardScaler()
df_scaled = StandardScaler().fit_transform(df)
print(df_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [56]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)
df_pca

array([[-0.33701255,  1.03001696, -0.51059724],
       [-0.320358  ,  1.03017243, -0.51077964],
       [ 2.31396781,  1.62875552, -0.56015115],
       ...,
       [ 0.32564298, -2.29232316,  0.44363844],
       [-0.15249203, -2.01375519,  0.23803376],
       [-0.29239486,  0.82476667, -0.27668694]])

In [57]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data = df_pca, columns = ['PC1', 'PC2', 'PC3'], index = crypto_df.index)

pcs_df

,PC1,PC2,PC3
42,-0.337013,1.030017,-0.510597
404,-0.320358,1.030172,-0.510780
1337,2.313968,1.628756,-0.560151
BTC,-0.149838,-1.326579,0.192362
ETH,-0.151890,-2.059342,0.376454
...,...,...,...
ZEPH,2.478093,0.717510,0.230748
GAP,-0.335059,1.029908,-0.510610
BDX,0.325643,-2.292323,0.443638
ZEN,-0.152492,-2.013755,0.238034


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [58]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

elbow_data = {"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [59]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.fit_predict(pcs_df)

In [62]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pcs_df,coins_name_df],axis = 1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"]= model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.337013,1.030017,-0.510597,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320358,1.030172,-0.510780,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.313968,1.628756,-0.560151,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.149838,-1.326579,0.192362,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.151890,-2.059342,0.376454,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.171160,-1.093486,0.014193,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.386394,1.179310,-0.444919,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.152069,-2.208186,0.438570,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.150334,-2.059436,0.376444,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.152491,-2.013755,0.238034,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [63]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [64]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(sortable=True)

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class]

In [65]:
# Print the total number of tradable cryptocurrencies.
print(clustered_df.count) 

<bound method DataFrame.count of         Algorithm ProofType  TotalCoinsMined TotalCoinSupply       PC1  \
42         Scrypt   PoW/PoS     4.199995e+01              42 -0.337013   
404        Scrypt   PoW/PoS     1.055185e+09       532000000 -0.320358   
1337          X13   PoW/PoS     2.927942e+10    314159265359  2.313968   
BTC       SHA-256       PoW     1.792718e+07        21000000 -0.149838   
ETH        Ethash       PoW     1.076842e+08               0 -0.151890   
...           ...       ...              ...             ...       ...   
ZEPH      SHA-256      DPoS     2.000000e+09      2000000000  2.478093   
GAP        Scrypt   PoW/PoS     1.493105e+07       250000000 -0.335059   
BDX   CryptoNight       PoW     9.802226e+08      1400222610  0.325643   
ZEN      Equihash       PoW     7.296538e+06        21000000 -0.152492   
XBC        Scrypt       PoS     1.283270e+05         1000000 -0.292395   

           PC2       PC3     CoinName  Class  
42    1.030017 -0.510597      4

In [66]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply","TotalCoinsMined"]])
print(scaled)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [67]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_df = pd.DataFrame(    
    data = scaled,columns = ["TotalCoinSuppy","TotalCoinsMined"], index = clustered_df.index
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
new_df = pd.concat([new_df,clustered_df['CoinName']],axis =1)

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = pd.concat([new_df,clustered_df["Class"]],axis = 1)

plot_df.head(10)

,TotalCoinSuppy,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [82]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
clustered_df.hvplot.scatter(
    x="TotalCoinsMined", 
    y="TotalCoinSupply",
    hover_cols=['CoinName'],
    loglog=True,
    ylim=(80,10000000000000000),
    by='Class') 

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)